In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

import flask
import pandas as pd
import time
import os

import sqlite3

In [ ]:
server = flask.Flask('app')
server.secret_key = os.environ.get('secret_key', 'secret')

conn=sqlite3.connect('documents/homeworkarchive/p3d/zdat.sqlite')
c=conn.cursor()
c.execute('select * from zilloData;')

In [ ]:
df=pd.read_sql_query('select * from zilloData;',conn)

In [ ]:
app = dash.Dash('app', server=server)
app.scripts.config.serve_locally = False
server = app.server

app.layout = html.Div([
    html.H2('Zillo Data'),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in df['RegionName']],
        value='Minneapolis'
    ),
    dcc.Graph(id='my-graph')
], className="container")

@app.callback(Output('my-graph', 'figure'),
              [Input('my-dropdown', 'value')])

def update_graph(selected_dropdown_value):
    dff = df[df['RegionName'] == selected_dropdown_value]
    return {
        'data': [{
            'x': dff.Date,
            'y': dff.Number_of_Listings,
            'line': {
                'width': 3,
                'shape': 'spline'
            }
        }],
        'layout': {
            'margin': {
                'l': 30,
                'r': 20,
                'b': 30,
                't': 20
            }
        }
    }

if __name__ == '__main__':
    app.run_server()

In [ ]:
%tb